In [4]:
import mysql.connector
from tqdm import tqdm
import re

In [ ]:
conn = mysql.connector.connect(
    host = "*****",
    port = "****",
    user = "****",
    password = "****",
    database = "****"
)

cursor = conn.cursor(buffered=True)

In [3]:
cursor.execute("select tech_name from tech")
tech_names = cursor.fetchall()

for t in tech_names:
    print(t)


('.NET',)
('ABAP',)
('AI(인공지능)',)
('AIX',)
('Ajax',)
('Android',)
('Angular',)
('Apache',)
('API',)
('APM',)
('AR(증강현실)',)
('ArcGIS',)
('ASP',)
('ASP.NET',)
('AWS',)
('Azure',)
('BI 엔지니어',)
('Bootstrap',)
('C#',)
('C++',)
('CentOS',)
('CISO',)
('COBOL',)
('CPO',)
('CSS',)
('CSS3',)
('C언어',)
('Dapp',)
('DBA',)
('DBMS',)
('Delphi',)
('DevOps',)
('DID',)
('Directx',)
('Django',)
('DLP',)
('Docker',)
('DW',)
('Eclipse',)
('ECMAScript',)
('ElasticStack',)
('ERP',)
('ETL',)
('FAE',)
('Flask',)
('FLEX',)
('Flutter',)
('FPGA',)
('GCP',)
('GIS',)
('Git',)
('GM(게임운영)',)
('GoLang',)
('GraphQL',)
('Groovy',)
('Gulp',)
('H/W',)
('Hadoop',)
('HBase',)
('HTML',)
('HTML5',)
('HTTP',)
('IaaS',)
('iBATIS',)
('ICT컨설팅',)
('IDC',)
('IIS',)
('Ionic',)
('iOS',)
('IoT',)
('ISMS',)
('IT컨설팅',)
('Java',)
('Javascript',)
('Jenkins',)
('JPA',)
('jQuery',)
('JSP',)
('Kafka',)
('Keras',)
('Kotlin',)
('Kubernetes',)
('LabVIEW',)
('Linux',)
('Logstash',)
('Lucene',)
('MacOS',)
('MariaDB',)
('Matlab',)
('Maven',)
('MCU

In [1]:
conn.close()

NameError: name 'conn' is not defined

In [11]:
cursor.execute("select recruit_id, post_txt, post_url, comp_id from recruit")
recruit_results = cursor.fetchall()

cursor.execute("""
            select t.tech_id tech_id, t.tech_name keyword from tech t 
            union 
            select ts.tech_id tech_id, ts.syn_name keyword from tech_synonyms ts order by tech_id;
            """)
tech_results = cursor.fetchall()

tech_dict = {}

for tech_id, keyword in tech_results:
    if tech_id in tech_dict:
        tech_dict[tech_id].add(keyword.lower())  # 이미 있는 tech_id에 새로운 키워드 추가
    else:
        tech_dict[tech_id] = {keyword.lower()}  # 새로운 tech_id에 키워드 리스트 생성


for r in tqdm(recruit_results, total=len(recruit_results)):
    recruit_id = r[0]
    post_txt = r[1]
    post_url = r[2]
    comp_id = r[3]

    if post_txt is None:
        print(f"{recruit_id}'s post_txt is none")
        continue

    if comp_id is None:
        print(f"{recruit_id}'s comp_id is none")
        continue

    for tech_id, keywords in tech_dict.items():
        for keyword in keywords:
            if keyword in ["r", "c", "http"]:
                pattern = r'\b' + re.escape(keyword) + r'\b'
            else:
                pattern = re.escape(keyword)
            if re.search(pattern, post_txt.lower()):
                cursor.execute("insert ignore into recruit_tech (recruit_id, tech_id) values (%s, %s)", (recruit_id, tech_id))
                cursor.execute("select tech_cnt from comp_tech where comp_id=%s and tech_id=%s", (comp_id, tech_id))
                tech_cnt = cursor.fetchall()
                if tech_cnt:
                    cursor.execute("update comp_tech set tech_cnt=%s where comp_id=%s and tech_id=%s", (tech_cnt[0][0]+1, comp_id, tech_id))
                else:
                    cursor.execute("insert into comp_tech (comp_id, tech_id, tech_cnt) values (%s, %s, %s)", (comp_id, tech_id, 1))
                break
    

 57%|█████▋    | 457/797 [01:12<00:54,  6.23it/s]

463's post_txt is none


 72%|███████▏  | 577/797 [01:29<00:27,  7.89it/s]

578's comp_id is none


100%|██████████| 797/797 [01:49<00:00,  7.27it/s]


In [12]:
conn.commit()

In [13]:
conn.close()